In [9]:
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import KFold
import pickle

class HeartDiseaseApp:
    def __init__(self, root, models):
        self.root = root
        self.root.title("Heart Disease Prediction Application")

        # Labels and Entry widgets for user input
        labels = ["Age (in numbers)", "Sex (0: female, 1: male)", "Chest pain type(0-3)", "Resting BP", "Serum Cholestoral (mg/dl)",
                  "Fasting blood sugar > 120 mg/dl", "Resting electrocardiographic results (0-2)",
                  "Maximum heart rate achieved", "Exercise induced angina", "Oldpeak",
                  "Slope of the peak exercise ST segment", "Number of major vessels (0-3)", "Thal: 0 = normal; 1 = fixed defect; 2 = reversible defect"]

        self.entries = []

        for i, label in enumerate(labels):
            ttk.Label(root, text=f"{label}: ").grid(row=i, column=0, sticky=tk.W, padx=10, pady=5)
            entry = ttk.Entry(root)
            entry.grid(row=i, column=1, padx=10, pady=5)
            self.entries.append(entry)

        # Prediction result label
        self.result_label = ttk.Label(root, text="Prediction: ")
        self.result_label.grid(row=len(labels), column=0, columnspan=2, pady=10)

        # Button for prediction
        self.predict_button = ttk.Button(root, text="Diagnosis Test Result", command=self.predict)
        self.predict_button.grid(row=len(labels) + 1, column=0, columnspan=2, pady=10)

        # Sample data
        ttk.Label(root, text="Sample data to fill:").grid(row=len(labels) + 2, column=0, columnspan=2, pady=5)
        ttk.Label(root, text="52 1 2 172 199 1 1 162 0 0.5 2 0 3 => Person has Heart Disease").grid(
            row=len(labels) + 3, column=0, columnspan=2, pady=5)

        self.models = models

    def predict(self):
        try:
            # Collect user input
            input_data = [float(entry.get()) for entry in self.entries]

            # Apply each model to make predictions
            predictions = []
            for model in self.models:
                feature_names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
                # Reshape the array as we are predicting for one instance
                input_data_df = pd.DataFrame([input_data], columns=feature_names)

                # Make predictions
                prediction = model.predict(input_data_df)
                predictions.append(prediction[0])

            # Display results
            result_text = "The person has Heart Diseases" if any(predictions) else "The person has not Heart Diseases"
            self.result_label.config(text=f"Prediction: {result_text}")

        except ValueError:
            messagebox.showerror("Error", "Please enter valid numeric values for all input fields.")

if __name__ == "__main__":
    # dataset
    df = pd.read_csv('heart.csv')

    # target as prediction
    X_svm = df.drop('target', axis=1)
    y_svm = df['target']

    # Train an SVM model with 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    svm_models = []

    for train_index, test_index in kf.split(X_svm, y_svm):
        X_train, X_test = X_svm.iloc[train_index], X_svm.iloc[test_index]
        y_train, y_test = y_svm.iloc[train_index], y_svm.iloc[test_index]

        svm_model = SVC(kernel='linear')
        svm_model.fit(X_train, y_train)
        svm_models.append(svm_model)

    # Save the last SVM model using pickle 
    with open('trained_svm_model.pkl', 'wb') as model_file:
        pickle.dump(svm_models[-1], model_file)

    # Tkinter application for heart disease prediction 
    root_svm = tk.Tk()
    app_svm = HeartDiseaseApp(root_svm, svm_models)
    root_svm.mainloop()
